In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import Bio
from Bio.Seq import Seq

In [3]:
def smith_waterman(ref_seq,query,match,mismatch,W1):

    #Defining matrix (n+1)(m+1)
    H = np.zeros([len(ref_seq) + 1,len(query) + 1]) #Matrix of zeros length(ref_seq+1)<-rows x length(query+1)<-columns
    zero = 0
    #The martrix will have the referance series down the left and the query along the top
    #Defining scoring matrix
    for i in range(1,len(ref_seq) + 1): #For 'i' in every 'row'<-'letter' of the referance sequence
        for j in range(1,len(query) + 1):#For 'j' in every 'column'<-'letter' of the query
            if ref_seq[i-1] == query[j-1]: #if the letter in referance matches the letter in query, i-1 to match H with sequence
                H[i,j] = H[i-1,j-1] + match #Fill i,j with the preceeding diagonal score + match score
            else:
                H[i,j] = max(H[i-1,j] - W1,H[i,j-1] - W1,H[i-1,j-1] + mismatch, zero) #Otherwise choose the max score from 0, mismatch, or gap

    #Isolating Max-value and creating empty aligned sequences
    ref_aligned = [] #empty sequence for referance aligned
    query_aligned = []#empty sequence for query aligned
    i, j = np.unravel_index(H.argmax(),H.shape) #Find the max value in the array
    markers = []#Add a '|' for matches '-' for gaps and '.' for mismatches
    smith_waterman_score=H[i,j] #Highest value in array corresponds with overall score
    while H[i,j] != 0: #Whilst the coordinates do not equal zero
        score = H[i,j] #The score is equal to max value at the beginning
        directions = [(H[i-1,j-1],H[i-1,j],H[i,j-1])]#Choose the max value from left, up or diagonal movement

        if score == H[i-1,j-1] + match and ref_seq[i-1] == query[j-1]: #Score equals preceeding diagonal + match and the letters are a match
            ref_aligned.append(ref_seq[i-1]) 
            query_aligned.append(query[j-1])
            markers.append('|')
            i, j = i - 1 , j - 1 
        
        elif score == H[i-1,j-1]+ mismatch and ref_seq[i-1]!=query[j-1]: #Score equals preceeding diagonal + mismatch and letters mismatch
            ref_aligned.append(ref_seq[i-1])
            query_aligned.append(query[j-1])
            markers.append('.')
            i , j = i - 1 , j - 1
        
        elif score == H[i-1,j] - W1:#Score equals the square above - the gap penalty, this corresponda to an insertion in referance sequence
            ref_aligned.append(ref_seq[i-1])
            query_aligned.append('-')
            markers.append('-')
            i, j = i - 1, j
        
        elif score == H[i, j- 1] - W1: #Score equals the square to the left - the gap penalty ^^^
            ref_aligned.append('-')
            query_aligned.append(query[j-1])
            markers.append('-')
            i , j = i , j - 1
       
   
    length = min(len(ref_seq),len(query)) #length of alignment can only be the minimum length of the two sequence
    similarity = (smith_waterman_score/(length*match))*100#Similairty is score/maximum match score 
    ref_aligned = ''.join(ref_aligned[::-1])#Reverse all list for proper visualisation 
    query_aligned = ''.join(query_aligned[::-1])
    markers=''.join(markers[::-1])
    print("Score:", smith_waterman_score)
    print("Query Sequence is",round(similarity),"%","similar to Referance Sequence")
    i = 0
    while i < len(ref_aligned):#while i is less than the length of the aligned sequence
        print('target:',ref_aligned[i:i+60])#print off 60 letters of each sequence and markers
        print('       ',markers[i:i+60])
        print('query: ',query_aligned[i:i+60])
        print()
        i += 60